In [1]:
import dgl
import torch as th
from torch import nn

Using backend: pytorch


In [2]:
net = dgl.nn.conv.GATConv(20, 10, 2)

In [35]:
adj = th.rand((40, 40))
u, v = th.nonzero(adj > 0.5, as_tuple=True)
g = dgl.graph((u, v))
h = th.rand((40, 20))
g.ndata['tmp'] = h
batch = dgl.batch([g for _ in range(32)])
batch_h = batch.ndata['tmp']

In [36]:
h_out, attn = net(g, h, get_attention =True)
attn.squeeze()

In [37]:
h_out, attn = net(batch, batch_h, get_attention = True)
batch.edata['_tmp_attn'] = attn
batch_list_attn = []

In [40]:
edges_per_g = batch.batch_num_edges()
edges_last_index = edges_per_g.cumsum(0)
edges_first_index = edges_last_index - edges_per_g[0]

In [41]:
attn_list = [attn[start:stop] for start, stop in zip(edges_first_index, edges_last_index)]

In [43]:
attn_list[0].

torch.Size([765, 2, 1])

In [24]:
[g.num_edges() for g in dgl.unbatch(batch)]

[<bound method DGLHeteroGraph.num_edges of Graph(num_nodes=40, num_edges=804,
       ndata_schemes={'tmp': Scheme(shape=(20,), dtype=torch.float32)}
       edata_schemes={})>,
 <bound method DGLHeteroGraph.num_edges of Graph(num_nodes=40, num_edges=804,
       ndata_schemes={'tmp': Scheme(shape=(20,), dtype=torch.float32)}
       edata_schemes={})>,
 <bound method DGLHeteroGraph.num_edges of Graph(num_nodes=40, num_edges=804,
       ndata_schemes={'tmp': Scheme(shape=(20,), dtype=torch.float32)}
       edata_schemes={})>,
 <bound method DGLHeteroGraph.num_edges of Graph(num_nodes=40, num_edges=804,
       ndata_schemes={'tmp': Scheme(shape=(20,), dtype=torch.float32)}
       edata_schemes={})>,
 <bound method DGLHeteroGraph.num_edges of Graph(num_nodes=40, num_edges=804,
       ndata_schemes={'tmp': Scheme(shape=(20,), dtype=torch.float32)}
       edata_schemes={})>,
 <bound method DGLHeteroGraph.num_edges of Graph(num_nodes=40, num_edges=804,
       ndata_schemes={'tmp': Scheme(shape=

In [44]:
 def forward_attn_scores(net, g):
    
    
    attn_tmp = []
    #### extract feature vector concat embeddings with handcrafted features
    emb = self.emb(g.ndata['residues']).view(-1, self.emb_size)
    emb2 = self.emb2(g.ndata['secondary']).view(-1, self.emb2_size)
    h, e = g.ndata['features'], g.edata['features']
    h = torch.cat([emb, emb2, h], dim=1)

    ####### first layer  block #############
    h, e, attn = self.gat_first.gat(g, h, e, get_attention = True)
    h, e = h.flatten(1), e.flatten(1)
    h, e = self.bn_first(h), self.bn_edge(e)
    h = F.leaky_relu(h)
    h = self.dp_first(h)
    attn_tmp.append(attn)
    ###### loop layer block #############
    for block in self.GATBlocks:
        h, e, attn = block.gat(g, h, e, get_attention = True)
        attn_tmp.append(attn)
    ###### last layer block #############
    h, e, attn = self.gat_last.gat(g, h, e, get_attention = True)
    attn_tmp.append(attn)
    
    return attn_tmp
